## **Face Recognition using Deep Learning CNN**

<html><img src="https://thinkingneuron.com/wp-content/uploads/2020/10/CNN-case-study.png" alt="Convolution step in CNN"></html>

**Convolutional Neural Network (CNN)**
- Focuses on one portion of the image at a time and scanning the whole image.
- It boils down every image as a vector of numbers, which can be learned by the fully connected Dense Layers of ANN.

<html><img src="https://thinkingneuron.com/wp-content/uploads/2020/10/CNN-face-recognition-case-study.png"></html>

**Dataset:** 
- Containes cropped face images of 16 people divided into Training and Testing,
- Will,
    - Train the CNN model using the images in the **Training folder**
    - Test the model using the unseen images from the **Testing folder**, to check if the model is able to recognise the face number of the unseen images or not.

In [1]:
## Deep Learning CNN model to recognize face
'''####### IMAGE PRE-PROCESSING for TRAINING and TESTING data #######'''

# Specifying the folder where images are present
TrainingImagePath='../Datasets/CNN/Face Images/Final Training Images'

In [2]:
!pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable


In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Defining pre-processing transformations on raw images of training data
# These hyper parameters helps to generate slightly twisted versions
# of the original images, which leads to a better model, since it learns
# on the bad and good mix of images

train_datagen=ImageDataGenerator(
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

# Defining pre-processing transformations on raw images of testing data
# No transformations are done on the testing images
test_datagen=ImageDataGenerator()

# Generating the Training Data
training_set=train_datagen.flow_from_directory(
    TrainingImagePath,
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical')

# Generating the Testing Data
test_set=test_datagen.flow_from_directory(
    TrainingImagePath,
    target_size=(64,64),
    class_mode='categorical')

# Printing class labels for each class
test_set.class_indices

Found 244 images belonging to 16 classes.
Found 244 images belonging to 16 classes.


{'face1': 0,
 'face10': 1,
 'face11': 2,
 'face12': 3,
 'face13': 4,
 'face14': 5,
 'face15': 6,
 'face16': 7,
 'face2': 8,
 'face3': 9,
 'face4': 10,
 'face5': 11,
 'face6': 12,
 'face7': 13,
 'face8': 14,
 'face9': 15}

## Creating a mapping for index and a face names

- The above class_index dictionary has face names as keys and the numeric mapping as values.
- Swapping it, as classifier model will return the answer as the numeric mapping, while answer needs to get the face-name out of it.

**As its a multiclass problem,**
- Counting the number of unique faces, as it'll be used as the number of output layer of fully connected ANN classifier.

In [5]:
# Creating lookup table for all faces
# Class_indices have the numeric tag for each face
TrainClasses=training_set.class_indices

# Storing the face and numeric tag
ResultMap={}

for faceValue, faceName in zip(TrainClasses.values(), TrainClasses.keys()):
    ResultMap[faceValue]=faceName

In [6]:
# Storing face maps for future reference
import pickle
with open("ResultMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)

#### This model will give answer as a numeric tag

In [7]:
# This mapping will help to get the corresponding face name for it
print("Mapping of Face and its ID", ResultMap)

# The number of neurons for the output layer is equal to the number of faces
OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

Mapping of Face and its ID {0: 'face1', 1: 'face10', 2: 'face11', 3: 'face12', 4: 'face13', 5: 'face14', 6: 'face15', 7: 'face16', 8: 'face2', 9: 'face3', 10: 'face4', 11: 'face5', 12: 'face6', 13: 'face7', 14: 'face8', 15: 'face9'}

 The Number of output neurons:  16


## **Creating the CNN face recognition model**

Model has:
- 2 hidden layers of convolution
- 2 hidden layers of max pooling
- 1 layer of flattening
- 1 output layer with 16 neurons (one for each face)

Important hyperparameters:

- **Filters**=32: 
    - This number indicated how many filters we are using to look at the image pixels during the convolution step.
    - Some filters may catch sharp edges, some filters may catch color variants, some filters may catch outliers, etc.
    - In the first layer, the number of filter=32 is commonly used, the increasing the power of 2, i.e, next layer is 64, next is 128 and so on.

- **kernel_size**=(5,5):
    - Indicated the size of sliding window during convolution, using 5*5 pixels here

- **strides**=(1,1):
    - How fast/slow should the sliding window move during convolution.
    - Using 1*1 (lowest setting)
    - Meaning, sliding the convolution window of 5*5 (kernel_size) by 1 pixel in the x-axis and 1 pixel in the y-axis until the whole thing is scanned.

- **input_shape**=(64,64,3):
    - Images in the dataset were compressed into 64*64 during pre-processing
    - Images are matrix of RGB (3 colors) codes.
    - This expected shape is (64,64,3) as in 3 arrays of (64,64), one for RGB colors each

- **activation**='relu':
    - This specifies the activation function for each calculations inside each neuron.
    - Thus choosing values like 'relu', 'tanh', 'sigmoid', etc.

- **optimizer**='adam':
    - This parameter helps to find the optimum values for each weight in the neural network.
    - 'adam' is one of the most useful optimizers, another one is 'misprop'

- **steps_per_epochs**=8:
    - Specifies how many rows will be passed to the Network, in one go after which the error calculation will begin and the neural network will start adjusting its weights based on the errors.
    - When all the rows are passed in the batches of 8 rows each as specified in this parameter, then we call that 1-epoch, or one full data cycle.
- **Epochs**=60:
    - The same activity of adjusting weights continues for 60 times, as specified by this parameter.
    - In simple terms, the CNN looks at the full training data 60 times and adjusts its weights, thus tuned based on accuracy value.

In [9]:
# Creating CNN deep learning model
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense

'''Initializing the Convolutional Neural Network'''
classifier=Sequential()

### Step 1: **Convolution**
- Adding the first layer of CNN
- Using the format (64,64,3) as using TensorFlow backend
- It means 3 matrix of size (64*64) pixels representing Red, Green and Blue components of pixels 

In [10]:
classifier.add(Convolution2D(32, kernel_size=(5,5), strides=(1,1), input_shape=(64,64,3), activation='relu'))

c:\Users\Hp\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2: **Max Pooling**

In [11]:
classifier.add(MaxPool2D(pool_size=(2,2)))

Adding additional layer of Convolution for better accuracy

In [12]:
classifier.add(Convolution2D(64, kernel_size=(5,5), strides=(1,1), activation='relu'))

classifier.add(MaxPool2D(pool_size=(2,2)))

### Step 3: **Flattening**

In [13]:
classifier.add(Flatten())

### Step 4: **Fully connected Neural Network**

In [14]:
classifier.add(Dense(OutputNeurons, activation='softmax'))

### **Compiling the CNN**

In [15]:
classifier.compile(loss='categorical_crossentropy', optimizer='adam', metrics=["accuracy"])

#### Measuring time taken by model to train

In [23]:
import time

startTime=time.time()

# Starting the model training
classifier.fit(
    training_set,
    steps_per_epoch=8,
    epochs=60,
    validation_data=test_set,
    validation_steps=4)

endTime=time.time()

print("Total Time Taken: ", round((endTime-startTime)/60), "Minutes")

Epoch 1/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 103ms/step - accuracy: 1.0000 - loss: 0.0025 - val_accuracy: 1.0000 - val_loss: 1.7192e-04
Epoch 2/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 1.5536e-04


c:\Users\Hp\AppData\Local\Programs\Python\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


Epoch 3/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - accuracy: 0.9952 - loss: 0.0071
Epoch 4/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 4.8796e-04
Epoch 5/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 63ms/step - accuracy: 0.9897 - loss: 0.0108 - val_accuracy: 0.9828 - val_loss: 0.0305
Epoch 6/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 85ms/step - accuracy: 1.0000 - loss: 0.0045 - val_accuracy: 1.0000 - val_loss: 0.0014
Epoch 8/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0015
Epoch 9/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 1.0000 - loss: 0.0089
Epoch 10/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 1.0637e-04
Epoch 11/60
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 75ms/step - accuracy: 1.0000 - loss: 0.0020 - 

## **Testing CNN on unseen images**

In [25]:
import numpy as np
from keras.preprocessing import image

ImagePath='../Datasets/CNN/Face Images/Final Testing Images/face4/3face4.jpg'

test_image=image.load_img(ImagePath, target_size=(64,64))
test_image=image.img_to_array(test_image)

test_image=np.expand_dims(test_image, axis=0)

result=classifier.predict(test_image, verbose=0)

print('Prediction is: ', ResultMap[np.argmax(result)])

Prediction is:  face4


## Conclusion

Case Study of CNN for face recognition completed.